### this is my notebook for trying to figure out how to make medical history information into something useful

#### first we import libraries and data

In [5]:
import pandas as pd
import numpy as np

import os
import re
from collections import Counter

import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import sklearn.metrics
import xgboost as xgb

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import random as rn

from IPython.display import clear_output

from string import punctuation


In [6]:
#use this cell when working online
path = '/floyd/home/ed-triage'
data_path = '/floyd/home/data'

In [7]:
#use this cell when working from home
path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage'
data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/combo'

In [8]:
clin = pd.read_csv(data_path + '/complete_clean_combo_data.csv', index_col = 0,low_memory = False)

In [9]:
clin.shape

(165433, 121)

#### then we load up my helper functions

#### then we clean up the data in the medical history column

In [10]:
history = list(clin['MedicalHistory'].str[16:].astype('str'))

In [89]:
#this is to clean the text in the medical history column (a little bit)
def fixhx(s):
    s =s.replace('<LT><LF>','')
    s = s.replace('<LT>LF>','')
    s = s.replace('<LF><LF>','')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = re.sub(r'\bOR\b', 'operation', s)
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('h/o', 'history of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    
    s = re.sub(r'\bhrs\b', 'hours', s)
    s = re.sub(r'\bwks\b', 'weeks', s)
    s = re.sub(r'\byrs\b', 'years', s)
    
    s = re.sub(r'\bhx\b', 'history', s)
    s = re.sub(r'\bdx\b', 'diagnosis', s)
    
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('lmp', 'last menstrual period')
    s = re.sub(r'\biud\b', 'intrauterine contraceptive device', s)
    s = re.sub(r'\biucd\b', 'intrauterine contraceptive device', s)
    s = re.sub(r'\bvb\b', 'vaginal bleed', s)
    s = s.replace('w/', 'with')
    s = s.replace('@', ' at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\br\s*\/\s*o','rule out',s)
    s = re.sub(r'\bc\s*\/\s*o','complains of',s)
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\blt\b','left ',s)
    s = re.sub(r'\brt\b','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics')
    s = re.sub(r'\bhtn\b', 'hyptertension', s)
    s = re.sub(r'\btyl\b', 'tylenol', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant  ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant  ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    s = re.sub('(?<=[a-zA-Z])-(?=[a-zA-Z])', ' ', s)  #I think this is mean to manage dashes, but I have done this a couple of other ways
    s = re.sub(r'\bwic\b', 'walk in clinic ',s)
    s = re.sub(r'\bcabg\b', 'coronary artery bypass ',s)
    s = re.sub(r'\bgerd\b', 'gastroesophageal reflux ',s)
    s = re.sub(r'\badhd\b', 'attention deficit disorder ',s)
    s = re.sub(r'\badd\b', 'attention deficit disorder ',s)
    s = re.sub(r'\bcopd\b', 'chronic obstructive pulmonary disease ',s)
    s = re.sub(r'\bems\b', 'emergency medical services ',s)
    s = re.sub(r'\bmvc\b', 'motor vehicle collision  ',s)
    s = re.sub(r'\bgp\b', 'family doctor  ',s)
    s = re.sub(r'\bloc\b', 'loss of consciousness ',s)
    s = re.sub(r'\bsts\b', 'states  ',s)
    s = re.sub(r'\bmvc', 'motor vehicle collision ',s)
    s = re.sub(r'\bmva', 'motor vehicle collision ',s)
    s = re.sub(r'\bnsr\b', 'normal sinus rhythm ',s)
    s = re.sub(r'\bbp\b', 'blood pressure ',s)
    s = re.sub(r'\betoh\b', 'alcohol ',s)
    s = re.sub(r'\bhtn\b', 'hypertension ',s)
    s = re.sub(r'\btia\b', 'transient ischemic attack ',s)
    s = re.sub(r'\biddm\b', 'type one diabetes ',s)
    s = re.sub(r'\bniddm\b', 'type two diabetes ',s)
    s = re.sub(r'\biddm\b', 'type one diabetes ',s)
    s = re.sub(r'\bdmi\b', 'type one diabetes ',s)
    s = re.sub(r'\bdm i\b', 'type one diabetes ',s)
    s = re.sub(r'\bdm1\b', 'type one diabetes ',s)
    s = re.sub(r'\bniddm\b', 'type two diabetes ',s)
    s = re.sub(r'\bdmii\b', 'type two diabetes ',s)
    s = re.sub(r'\bdm2\b', 'type two diabetes ',s)
    s = re.sub(r'\bdm ii\b', 'type two diabetes ',s)
    s = re.sub(r'\bcrf\b', 'chronic renal failure ',s)
    s = re.sub(r'\bckd\b', 'chronic renal failure ',s)
    s = re.sub(r'\bchf\b', 'congestive heart failure',s)
    s = re.sub(r'\bcad\b', 'coronary artery disease ',s)
    s = re.sub(r'\bmi\b', 'myocardial infarction  ',s)
    s = re.sub(r'\bsbo\b', 'small bowel obstruction ',s)
    s = re.sub(r'\blbo\b', 'large bowel obstruction ',s)
    s = re.sub(r'\baaa\b', 'abdominal aortic aneurysm',s)
    s = re.sub(r'\bocd\b', 'obsessive compulsive disorder',s)
    s = re.sub(r'\bpvd\b', 'peripheral vascular disease',s)
    s = re.sub(r'\bdvt\b', 'deep vein thrombosis',s)
    s = re.sub(r'\bibs\b', 'irritable bowel syndrome',s)
    s = re.sub(r'\basd\b', 'atrial septal defect',s)
    s = re.sub(r'\bvsd\b', 'ventricular septal defect',s)
    s = re.sub(r'\bpci\b', 'percutaneous coronary intervention',s)
    s = re.sub(r'\brsv\b', 'respiratory syincitial virus',s)
    s = re.sub(r'\btbi\b', 'traumatic brain injury',s)
    s = re.sub(r'\btb\b', 'tuberculosis',s)
    s = re.sub(r'\bacs\b', 'acute coronary syndrome',s)
    s = re.sub(r'\boa\b', 'osteoarthritis',s)
    s = re.sub(r'\bdnr\b', 'do not resuscitate',s)
    s = re.sub(r'\bpid\b', 'pelvic inflammatory disease',s)
    s = re.sub(r'\bpe\b', 'pulmonary embolus',s)
    s = re.sub(r'\babd\b', 'abdomen',s)
    s = re.sub(r'\bcva\b', 'stroke',s) 
    s = re.sub(r'\bca\b', 'cancer ',s)
    s = re.sub(r'and/or', 'or', s)
    s = re.sub(r'\bneg', 'negative', s)
    s = re.sub(r's/b', 'sent by', s)
    s = re.sub(r'nausea\s*\/\s*vomiting', 'nausea or vomiting', s)
    s = re.sub(r';',',',s)
    s = re.sub(r'aaox3', 'alert and oriented', s)
    s = re.sub(r'csm', ' circulation, sensation and movement ',s)
    s = re.sub(r'\bsz\b', ' seizure ',s)
    s = re.sub(r'hgb', 'hemoglobin', s)
    s = re.sub(r'\basa\b', 'aspirin', s)
    s = re.sub(r'\butd\b', 'up to date', s)
    s = re.sub(r'\brx\b', 'treatment', s)
    s = re.sub(r'\bnpo\b', 'nothing by mouth', s)
    s = re.sub(r'd/c', 'discharge', s)
    s = re.sub(r'o2', 'oxygen', s)
    s = re.sub(r'iv', 'intravenous', s)
    s = re.sub(r'rxn', 'reaction', s)
    
    
    #managing punctuation
    s = s.replace("--", "-")
    s = s.replace('-', ' ')
    s = re.sub(' of ', ' ', s)
    s = re.sub(' to ', ' ', s)
    s = re.sub(' in ', ' ', s)
    s = re.sub(' at ', ' ', s)
    s = re.sub('\)', '', s)
    s = re.sub(r'\s+,',',', s)
    s = re.sub(r'\s+\.','.', s)

    puncts = '!"$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    s = s.translate(str.maketrans('','',puncts))
        
    #punctuation_spacer = str.maketrans({key: f"{key} " for key in ".,"})
    #s = s.translate(punctuation_spacer)
    
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    return s

In [90]:
replacements = (',','<LT>LF>', '.', ';', '/')
drop_list = ('"', '?', 'had', 'has', 'have', 'left', 'right', 'sided','weeks', 'history of', 'h/o','and', 'acute')   #added all but first 2 options may 30, 2020
split_hist = []
for item in history:
    for r in replacements:
        item = item.replace(r, '::')
        
    item = item.split('::')
    item = [re.sub(r"(\b[a-z]\b)\s([a-z])", r"\1-\2", s) for s in item]  #to handle a fib, c section, h pylori
    item = [re.sub(r'\(.*?\)\ *', '', s) for s in item]  #remove punctuation
    item = [re.sub(r'\b\d+\b', '', s) for s in item]  #remove numbers
    item = [re.sub(r'%', '', s) for s in item]  #remove percent sign
    item = [re.sub(r'\'', '',s) for s in item]  #remove '
    item = [re.sub(r'\b[0-9]\w+''', '', s) for s in item]  #remove words that start with numbers
    item = [re.sub(r'^[^\w#]*', '',s) for s in item]    #remove leading symbols except fracture
    item = [re.sub(r'^-\s*', '',s) for s in item] #remove leading dashes
    item = [s.strip('-') for s in item] #remove leading and trailing dashes

    
    item = [fixhx(s) for s in item]   #run the cleaning function above to get rid of some of the abbreviation
    
    
    for d in drop_list:
        item = [x.replace(d,'') for x in item]   #get rid of the words from the blacklist above
    
    item = [x.strip().lower() for x in item if x != 'nan']   #get rid of white spaces
    item = [x for x in item if len(x) > 2]
    item = list(filter(None, item))
    
    split_hist.append(item)
    
    
 #split hist is a list of lists, where each entry is the list of past medical history conditions for each patient

In [91]:
#split hist is a list of lists, where each entry is the list of past medical history conditions for each patient
split_hist[:20]

[['no significant medical history'],
 ['no significant medical history'],
 ['no significant medical history'],
 ['dialysis haemo',
  'type one diabetes',
  'chronic renal failure',
  'heart problem',
  'high cholesterol'],
 ['fibromyalgia'],
 ['enlarged lintravenouser'],
 ['anxiety',
  'autistic',
  'attention deficit disorder',
  'depression',
  'asthma',
  'celiac'],
 ['no significant medical history'],
 ['hypothyroid'],
 ['anxiety'],
 ['no significant medical history'],
 ['hypertension'],
 ['arthritis', 'chronic obstructintravenouse pulmonary disease'],
 ['itp', 'wiscott aldridge syndrome', 'eczema'],
 ['suicidal attempts', 'depression'],
 ['high cholesterol',
  'hypertension',
  'type two diabetes',
  'coronary artery bypass',
  'knee surgery years ago'],
 ['sepsis',
  'urinary tract infection',
  'cancer lungs',
  'cancer bronchus',
  'hypothyroid',
  'glaucoma cataract',
  'cataract repair',
  'kidney disease',
  'anemia'],
 ['alcoholism'],
 ['gastroesophageal reflux', 'high chol

In [76]:
#my hideous way of limiting each diagnosis to 3 words long
split_hist2 = []
for item in split_hist:
    item2 = []
    for dx in item:
        if dx != 'no significant medical history':
            dx = re.sub(r"(\b[b-z]\b)\s([a-z])", r"\1-\2", dx)
            new_dx = ' '.join(dx.split()[:3])
            item2.append(new_dx)
        else:
            item2.append(dx)
    split_hist2.append(item2)

In [77]:
#I've processed it so each diagnosis is no more than 3 words long
split_hist2[:20]

[['no significant medical history'],
 ['no significant medical history'],
 ['no significant medical history'],
 ['dialysis haemo',
  'type one diabetes',
  'chronic renal failure',
  'heart problem',
  'high cholesterol'],
 ['fibromyalgia'],
 ['enlarged lintravenouser'],
 ['anxiety',
  'autistic',
  'attention deficit disorder',
  'depression',
  'asthma',
  'celiac'],
 ['no significant medical history'],
 ['hypothyroid'],
 ['anxiety'],
 ['no significant medical history'],
 ['hypertension'],
 ['arthritis', 'chronic obstructintravenouse pulmonary'],
 ['itp', 'wiscott aldridge syndrome', 'eczema'],
 ['suicidal attempts', 'depression'],
 ['high cholesterol',
  'hypertension',
  'type two diabetes',
  'coronary artery bypass',
  'knee surgery years'],
 ['sepsis',
  'urinary tract infection',
  'cancer lungs',
  'cancer bronchus',
  'hypothyroid',
  'glaucoma cataract',
  'cataract repair',
  'kidney disease',
  'anemia'],
 ['alcoholism'],
 ['gastroesophageal reflux', 'high cholesterol'],
 

### exploring different diagnoses

In [78]:
len(split_hist2)

165433

In [79]:
diagnoses = Counter()
for hx in split_hist2:   #note that this is done with my split modified split hist
    for item in hx:
        diagnoses[item] += 1

#gonna adapt the vocabulary to contain bi grams and trigrams
vocab = Counter()
for hx in split_hist2:   #note that this is done with my split modified split hist
    for item in hx:
        vocab[item] += 1
        word_list = item.split()
        if len(word_list) > 1:
            for word in word_list:
                vocab[word] += 1
                if len(word_list) == 3:
                    vocab[' '.join(word_list[:2])] +=1
                    vocab[' '.join(word_list[1:])] +=1
        


In [81]:
len(diagnoses), len(vocab)

(19315, 34442)

In [82]:
vocab.most_common(25)

[('history', 74042),
 ('no', 73980),
 ('medical', 73744),
 ('significant', 73725),
 ('no significant medical history', 73713),
 ('type two', 38613),
 ('two diabetes', 38535),
 ('high', 28211),
 ('hypertension', 21929),
 ('cholesterol', 20857),
 ('blood pressure', 20539),
 ('diabetes', 20163),
 ('high cholesterol', 19840),
 ('high blood', 19730),
 ('type', 16813),
 ('two', 12914),
 ('type two diabetes', 12844),
 ('type one', 11517),
 ('one diabetes', 11496),
 ('hyptertension', 7948),
 ('blood', 7564),
 ('pressure', 7209),
 ('asthma', 7057),
 ('high blood pressure', 6570),
 ('depression', 6528)]

In [83]:
diagnoses.most_common(25)

[('no significant medical history', 73713),
 ('hypertension', 21678),
 ('high cholesterol', 19774),
 ('type two diabetes', 12844),
 ('hyptertension', 7806),
 ('asthma', 6986),
 ('high blood pressure', 6570),
 ('depression', 6437),
 ('thyroid', 4958),
 ('anxiety', 4913),
 ('hypothyroid', 3987),
 ('type one diabetes', 3832),
 ('gastroesophageal reflux', 3428),
 ('diabetes', 3263),
 ('acid reflux', 2422),
 ('arthritis', 2230),
 ('myocardial infarction', 2020),
 ('anemia', 1710),
 ('stroke', 1575),
 ('chronic obstructintravenouse pulmonary', 1533),
 ('high chol', 1285),
 ('congestintravenouse heart failure', 1189),
 ('schizophrenia', 1155),
 ('gout', 1128),
 ('seizure', 1066)]

In [84]:
sorted(diagnoses.most_common())[600:625]

[('alzheiimers', 1),
 ('alzheimer', 4),
 ('alzheimers', 199),
 ('alzheimers brain aneurysm', 1),
 ('alzheimers difficulty putting', 1),
 ('alzheimersneoplasm breast', 1),
 ('alzheimerz', 1),
 ('alzheimsers', 1),
 ('alzhemier', 1),
 ('alzhemiers', 3),
 ('alzhemirs', 1),
 ('alzhiemer', 2),
 ('alzhiemers', 2),
 ('alzhimer', 1),
 ('alzhimers', 1),
 ('alzhmier', 1),
 ('alzhmiers', 1),
 ('alziemers', 1),
 ('amaloid heart disese', 1),
 ('ambulates with cane', 1),
 ('ambutated leg', 1),
 ('amd', 1),
 ('amemia', 1),
 ('amenia', 10),
 ('amenorrhea', 1)]

In [85]:
diagnoses.most_common()[:-20-1:-1]

[('past history utis', 1),
 ('prior unknown heart', 1),
 ('kidney stone urinary', 1),
 ('iron med', 1),
 ('disc injury', 1),
 ('bariatric sx', 1),
 ('kideny transplant', 1),
 ('mikes on march', 1),
 ('abaltion st', 1),
 ('femur fracture three', 1),
 ('c6 stenosis', 1),
 ('list', 1),
 ('unspecified heart disease', 1),
 ('type ii diabetes', 1),
 ('paresthesia', 1),
 ('demyleanating neuropathy', 1),
 ('carotid sx', 1),
 ('tetnts', 1),
 ('legs paralyed', 1),
 ('currently being assessed', 1)]

In [86]:
vocab.most_common()[:-20-1:-1]

[('past history utis', 1),
 ('prior unknown heart', 1),
 ('kidney stone urinary', 1),
 ('iron med', 1),
 ('disc injury', 1),
 ('kideny transplant', 1),
 ('mikes on march', 1),
 ('abaltion', 1),
 ('abaltion st', 1),
 ('femur fracture three', 1),
 ('unspecified heart disease', 1),
 ('type ii diabetes', 1),
 ('demyleanating', 1),
 ('demyleanating neuropathy', 1),
 ('carotid sx', 1),
 ('tetnts', 1),
 ('paralyed', 1),
 ('legs paralyed', 1),
 ('currently being assessed', 1),
 ('bilateral lower leg', 1)]

In [87]:
## how am I gonna find and deal with all the 
abbrevs = []
for item, num in vocab.items():
    if len(item) <=3:
        abbrevs.append((item, num))


In [88]:
sorted(abbrevs, key = lambda x: x[1], reverse = True)

[('no', 73980),
 ('two', 12914),
 ('one', 3960),
 ('low', 1015),
 ('up', 922),
 ('bi', 494),
 ('sx', 471),
 ('hip', 468),
 ('x', 446),
 ('ago', 444),
 ('a', 411),
 ('d', 394),
 ('leg', 387),
 ('on', 310),
 ('fib', 308),
 ('the', 255),
 ('eye', 241),
 ('ear', 208),
 ('for', 207),
 ('c', 191),
 ('x2', 175),
 ('use', 172),
 ('see', 171),
 ('svt', 147),
 ('#', 144),
 ('gad', 139),
 ('arm', 137),
 ('hep', 128),
 ('x1', 108),
 ('b', 101),
 ('b12', 100),
 ('pre', 89),
 ('vit', 83),
 ('x3', 82),
 ('not', 81),
 ('non', 74),
 ('bil', 68),
 ('gi', 67),
 ('feb', 63),
 ('dec', 62),
 ('due', 57),
 ('by', 57),
 ('air', 56),
 ('itp', 54),
 ('may', 54),
 ('dm', 51),
 ('jan', 50),
 ('was', 47),
 ('oct', 47),
 ('as', 44),
 ('icd', 42),
 ('ddd', 39),
 ('gl', 37),
 ('nov', 36),
 ('sgx', 35),
 ('day', 35),
 ('old', 35),
 ('hi', 32),
 ('aug', 31),
 ('odd', 31),
 ('toe', 30),
 ('age', 30),
 ('hr', 30),
 ('x4', 29),
 ('is', 28),
 ('acl', 28),
 ('svd', 27),
 ('def', 25),
 ('sci', 24),
 ('cll', 24),
 ('rib', 22)

### this is getting much better.  gonne now do some work to make all the aaa's, c-sections, etc. collapse to a single diagnosis

In [566]:
#this function takes in a counter with word frequencies of similar words
#it returns the most common spelling as the "best spelling"

def get_best_spelling(word, dict_name):
    #print (word)
    max_value = -10
    best_spelling = 0
    for k,v in dict_name.items():
        if v > max_value:
            max_value = v
            best_spelling = k
    
    if max_value == 1:
        best_spelling = word
        
    return best_spelling 

In [643]:
#this is stolen from peter norvigs spellchecker
import re
from collections import Counter

#def words(text): return re.findall(r'\w+', text.lower())

WORDS = vocab

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

#### now i need to find a way to generate the frequency dict/counter to use to get the best spelling

In [644]:
#gonna start using the functions from peter norvig
#arbitrarily using one of my c-words:  'cataracts'

def poss_words(word):
    return list(set(list(known(edits1(word))) +  list(known(edits2(word))) + [word]))

In [645]:
print (poss_words('c-section'))
print (poss_words('c section'))
print (poss_words('cataract'))
print (poss_words('crohns'))
print(poss_words('c-diff'))

['c-sction', 'c-cesection', 'csection', 'disection', 'section', 'c-sections', 'csections', 'c-section', 'resection']
['c section', 'csection', 'disection', 'section', 'csections', 'resection']
['caatracts', 'cataract', 'cateract', 'catarract', 'cataracts', 'cataracs', 'caterac', 'catarac', 'catharact', 'catarct', 'catract', 'catracts']
['chronhs', 'cons', 'crohns', 'conns', 'chrohns', 'crohn', 'chrons', 'croups', 'crohs', 'irons', 'cronhs', 'chrns', 'chronns']
['diff', 'c-dif', 'c-diff', 'cdiffe', 'cdif', 'cdiff']


this function takes the diagnoses counter and generates a dictionary of possible spellings for each first word > 5 letters, beginning with the chosen letter

In [646]:
def first_letter_counter(letter, diagnosis_counter = diagnoses, word_length=5):
    collected_words = [word for word in sorted(diagnosis_counter.keys()) if word[0] == letter]
    
    let_counter = Counter(collected_words)
    
    first_words = []
    for word in let_counter.keys():
        first_word =  word.split()[0]
        first_words.append(first_word) 
        
    fw_counter = Counter(first_words)
    
    spell_dict = {}
    for first_word in first_words:
        if len(first_word) > word_length:
            poss_spells = poss_words(first_word)
            word_count = dict()
            for spell in poss_spells:
                word_count[spell] = fw_counter[spell]
            spell_dict[first_word] = word_count
        
    return collected_words, spell_dict

In [647]:
#this should have the same results as the cell by cell implementation with the letter c
%time c_collected_words, c_spell_dict = first_letter_counter('c', vocab, 4)

CPU times: user 4min 54s, sys: 479 ms, total: 4min 54s
Wall time: 4min 58s


In [648]:
for k,v in c_spell_dict.items():
    print (k,':')
    print (v)
    print (get_best_spelling(k,v))
    print ()

c-after :
{'c-after': 2, 'after': 0}
c-after

c-anca :
{'c-anca': 2, 'chance': 3}
chance

c-april :
{'april': 0, 'c-april': 1, 'sapril': 0}
c-april

c-bound :
{'c-bound': 1, 'bound': 0}
c-bound

c-carrier :
{'b-carrier': 0, 'c-carrier': 1, 'carrier': 3}
carrier

c-cesection :
{'c-cesection': 1, 'c-section': 70}
c-section

c-cestionsx :
{'c-cestionsx': 2}
c-cestionsx

c-cured :
{'cured': 1, 'c-cured': 1}
c-cured

c-deficiency :
{'s-deficiency': 0, 'b-dificiency': 0, 'c-deficiency': 1, 'v-deficiency': 0, 'd-deficency': 0, 'deficiency': 0, 'd-deficiency': 0, 'b-deficiency': 0, 'k-deficiency': 0, 's-deficieny': 0}
c-deficiency

c-dif :
{'b-def': 0, 'c-dif': 1, 'c-diff': 9, 'c-lip': 2, 'dif': 0, 'd-def': 0, 'cdif': 1, 'cdiff': 5}
c-diff

c-diff :
{'diff': 0, 'c-dif': 1, 'c-diff': 9, 'cdiffe': 1, 'cdif': 1, 'cdiff': 5}
c-diff

c-done :
{'cone': 3, 's-done': 0, 'c-done': 2, 'x-one': 0, 'c-on': 2, 'done': 0, 's-tone': 0}
cone

c-for :
{'for': 0, 'c-for': 1, 'h-for': 0, 'cfr': 1, 'cor': 1, 'x-f

cholesterol

cholesterolbipolar :
{'cholesterolbipolar': 1}
cholesterolbipolar

cholesterole :
{'cholesteron': 1, 'chholesterol': 1, 'choelesterol': 1, 'cholestyrol': 1, 'choleterol': 1, 'cholesterrol': 1, 'cholesteol': 1, 'cholesteroa': 1, 'cholersterol': 1, 'chlesterol': 1, 'chilesterol': 1, 'cholsterol': 1, 'cholesterols': 1, 'cholesteroll': 1, 'cholesetrol': 1, 'cbholesterol': 1, 'cholesteraol': 1, 'cholelsterol': 1, 'choesterol': 1, 'cholesterol': 35, 'cholestorol': 1, 'chollesterol': 1, 'scholesterol': 0, 'cholosterol': 1, 'choelsterol': 1, 'chpolesterol': 2, 'choloesterol': 1, 'choletserol': 1, 'cholesterolmi': 1, 'choleserol': 1, 'cholesterole': 1, 'chcolesterol': 1, 'cholestero': 1, 'cholesterl': 1, 'colesterol': 1, 'cholestrerol': 1, 'chlolesterol': 1, 'cholesteorl': 1, 'cholestreol': 1, 'cholestrol': 1, 'cholseterol': 1, 'choltesterol': 1, 'choresterol': 1, 'cholesteral': 1, 'cjholesterol': 1, 'sholesterol': 0}
cholesterol

cholesterolemia :
{'cholesterolemia': 1, 'cholester

correction :
{'collection': 2, 'correction': 1}
collection

cortical :
{'cervical': 32, 'cortical': 2, 'cortisol': 3, 'partical': 0}
cervical

cortisol :
{'cortisone': 1, 'cortical': 2, 'cortisol': 3}
cortisol

cortisone :
{'cortisone': 1, 'cortisol': 3}
cortisol

cosft :
{'csbt': 1, 'post': 0, 'costo': 2, 'soft': 0, 'coat': 3, 'lost': 0, 'cast': 3, 'cosft': 1, 'count': 2, 'cyst': 74, 'csvt': 2}
cyst

cosntipation :
{'contipation': 1, 'consptipation': 1, 'costipation': 1, 'constiapation': 1, 'constipaion': 1, 'constipation': 4, 'cosntipation': 1, 'constipiation': 1, 'connstipation': 1, 'constpation': 1, 'consitpation': 1}
constipation

costal :
{'aortal': 0, 'portal': 0, 'costo': 2, 'total': 0, 'costal': 3, 'distal': 0, 'crystal': 4}
crystal

costcochondritis :
{'costcochondritis': 1, 'costochondritis': 1}
costcochondritis

costiocongitis :
{'costiocongitis': 1}
costiocongitis

costipation :
{'contipation': 1, 'costipation': 1, 'constiapation': 1, 'consptipation': 1, 'constipaion': 1, 

In [649]:
len(c_spell_dict), len (c_collected_words)

(786, 3610)

looking good.  let's try with another letter for fun

In [657]:
%time h_diagnoses, h_spell_dict = first_letter_counter('h', vocab, 5)

CPU times: user 3min, sys: 130 ms, total: 3min
Wall time: 3min


In [658]:
#h_spell_dict should be shorter than h_diagnoses becuase it will only contain the first word of each diagnosis
len(h_diagnoses), len(set(h_diagnoses))

(2124, 2124)

In [659]:
for k,v in h_spell_dict.items():
    print (k,':')
    print (v)
    print (get_best_spelling(k,v))
    print ()

h-amputee :
{'h-amputee': 1, 'amputee': 0}
h-amputee

h-burn :
{'burn': 0, 'h-burn': 1}
h-burn

h-foot :
{'h-for': 1, 'h-from': 2, 'foot': 0, 'h-foot': 1}
h-from

h-fracture :
{'fracture': 0, 'h-fracture': 3, 't-fracture': 0}
h-fracture

h-from :
{'h-for': 1, 'from': 0, 'h-from': 2, 'h-foot': 1}
h-from

h-grip :
{'h-grip': 2}
h-grip

h-infection :
{'infection': 0, 'h-infection': 3}
h-infection

h-injury :
{'injury': 0, 'h-injury': 1}
h-injury

h-numbness :
{'h-numbness': 1, 'numbness': 0}
h-numbness

h-pain :
{'h-pain': 1, 'pain': 0}
h-pain

h-piloro :
{'h-piloro': 1, 'h-pylori': 5}
h-pylori

h-plyori :
{'h-pylorie': 1, 'h-plyorie': 1, 'hyplori': 1, 'h-plyori': 1, 'h-pylori': 5, 'h-pyloric': 1, 'hpylori': 2}
h-pylori

h-plyorie :
{'h-pylorie': 1, 'h-plyorie': 1, 'h-plyori': 1, 'h-pylori': 5, 'h-pyloric': 1}
h-pylori

h-pyloperationi :
{'hpyloperationi': 1, 'pyloperationi': 0, 'h-pyloperationi': 1}
h-pyloperationi

h-pylori :
{'h-pylorie': 1, 'h-plyorie': 1, 'hyplori': 1, 'h-plyori': 1,


hypertenson :
{'hypertension': 80, 'hypertnsion': 1, 'hypertesion': 1, 'hyperetension': 1, 'hyertension': 1, 'hypertenstion': 1, 'hypertyension': 1, 'hypertention': 3, 'hypetension': 1, 'hyoertension': 1, 'hypertenson': 1, 'hpertension': 1, 'hyptertension': 1}
hypertension

hypertenstion :
{'hypertension': 80, 'hypertnsion': 1, 'hypertesion': 1, 'hyperetension': 1, 'hypetention': 1, 'hyertension': 1, 'hypertenstion': 1, 'hpertention': 3, 'hypertyension': 1, 'hypertention': 3, 'hypetension': 1, 'hyoertension': 1, 'hypertenson': 1, 'hpertension': 1, 'hyptertension': 1}
hypertension

hypertention :
{'hypertension': 80, 'hypertnsion': 1, 'hypertesion': 1, 'hyperetension': 1, 'hypetention': 1, 'hyertension': 1, 'hypertenstion': 1, 'hpertention': 3, 'hypertyension': 1, 'hypertention': 3, 'hypetension': 1, 'hyoertension': 1, 'hypertenson': 1, 'hpertension': 1, 'hyptertension': 1}
hypertension

hypertesion :
{'hypertension': 80, 'hypertnsion': 1, 'hypertesion': 1, 'hyperetension': 1, 'hyerten

In [660]:
%time a_diagnoses, a_spell_dict = first_letter_counter('a', vocab, 5)

CPU times: user 3min 45s, sys: 255 ms, total: 3min 45s
Wall time: 3min 46s


In [661]:
len(a_spell_dict), len(set(a_diagnoses))

(657, 2773)

In [662]:
for k,v in a_spell_dict.items():
    print (k,':')
    print (v)
    print (get_best_spelling(k,v))
    print ()

aaortic :
{'arotic': 8, 'aaortic': 2, 'aortic': 60, 'gasrtic': 0, 'aotic': 2, 'aoric': 3, 'aortiv': 3, 'aortci': 2, 'aoctic': 3}
aortic

aarthritis :
{'arthirtis': 1, 'arhtritis': 1, 'arthrritis': 1, 'arthritis': 29, 'arthritism': 2, 'arthitis': 3, 'aarthritis': 1, 'arthiritis': 1, 'arthrirtis': 1, 'arthristis': 3, 'arthrits': 1, 'arthrisis': 1, 'arthtritis': 1, 'arthrtis': 1, 'aathritis': 1, 'athritis': 1, 'arthtitis': 1, 'arhthritis': 1, 'arrthritis': 1, 'artheritis': 1, 'arthrities': 1, 'arteritis': 1, 'artritis': 1, 'arthrittis': 1}
arthritis

aathritis :
{'arthirtis': 1, 'arhtritis': 1, 'arthrritis': 1, 'arthritis': 29, 'arthritism': 2, 'arthitis': 3, 'aarthritis': 1, 'arthiritis': 1, 'arthristis': 3, 'arthrirtis': 1, 'arthrits': 1, 'arthrisis': 1, 'arthtritis': 1, 'arthrtis': 1, 'aathritis': 1, 'athritis': 1, 'arthtitis': 1, 'sthritis': 0, 'arhthritis': 1, 'arrthritis': 1, 'artheritis': 1, 'arthrities': 1, 'arteritis': 1, 'artritis': 1, 'gatritis': 0, 'arthrittis': 1}
arthritis



anigina :
{'anginal': 5, 'anigina': 1, 'anagina': 1, 'angina': 10, 'anginia': 1}
angina

anioplasty :
{'angiopalsty': 1, 'angioplasy': 5, 'ioplasty': 0, 'angioplasty': 74, 'angioplastry': 3, 'angiopasty': 1, 'angiplasty': 4, 'anggioplasty': 1, 'angoioplasty': 1, 'agioplasty': 3, 'angiogplasty': 1, 'anioplasty': 1, 'anoloplasty': 1, 'angioplaty': 1, 'angioplast': 5}
angioplasty

anixety :
{'anxietyw': 1, 'anxiety': 23, 'anaxiety': 1, 'ansiety': 1, 'anxeity': 3, 'anxiey': 2, 'anixiety': 1, 'anciety': 1, 'anxitey': 1, 'naxiety': 0, 'anxety': 1, 'anixety': 1, 'anxietry': 1, 'aniety': 1, 'anxiet': 1, 'anxirty': 1, 'asnxiety': 1, 'axiety': 1, 'anxierty': 1, 'anxity': 1, 'amxiety': 1}
anxiety

anixiety :
{'anxietyw': 1, 'anxiety': 23, 'anaxiety': 1, 'ansiety': 1, 'anxeity': 3, 'anxiey': 2, 'anixiety': 1, 'anciety': 1, 'anxitey': 1, 'anxety': 1, 'naxiety': 0, 'anixety': 1, 'anxietry': 1, 'aniety': 1, 'anxiet': 1, 'asnxiety': 1, 'anxirty': 1, 'axiety': 1, 'anxierty': 1, 'anxity': 1, 'amxiety': 

### now gonna work on correcting the diagnoses

In [687]:
corrected_hist = []
for hist in bad_hist:  #list of diagnoses
    new_hist = []
    for dx in hist:  #each diagnosis (made of up to 3 words)
        words = dx.split()  #list of words in the diagnosis
        new_words = []
        for word in words:
            if word in h_spell_dict.keys():
                best_spelling = get_best_spelling(word, h_spell_dict[word])
                #print (best_spelling)
                new_words.append(best_spelling)
            else:
                new_words.append(word)
        #print (new_words)
        new_dx = ' '.join(new_words)
        new_hist.append(new_dx)
    corrected_hist.append(new_hist)
            
                
                


In [688]:
corrected_hist

[['no significant medical history'],
 ['no significant medical history'],
 ['no significant medical history'],
 ['dialysis haemo', 'iddm', 'crf', 'heart problem', 'high cholesterol'],
 ['fibromyalgia'],
 ['enlarged liver'],
 ['anxiety', 'autistic', 'adhd', 'depression', 'asthma', 'celiac'],
 ['no significant medical history'],
 ['hypothyroid'],
 ['anxiety'],
 ['no significant medical history'],
 ['hypertension'],
 ['arthritis', 'copd'],
 ['itp', 'wiscott aldridge syndrome', 'eczema'],
 ['suicidal attempts', 'depression'],
 ['high cholesterol', 'hypertension', 'niddm', 'cabg', 'knee surgery years'],
 ['sepsis',
  'urinary tract infection',
  'ca lungs',
  'ca bronchus',
  'hypothyroid',
  'glaucoma cataract',
  'cataract repair',
  'kidney',
  'anemia'],
 ['alcoholism'],
 ['gerd', 'high cholesterol'],
 ['hypertension',
  'tia',
  'osteoporosis',
  'diverticulitis',
  'high cholesterol',
  'anxiety',
  'depression',
  'aortic stenosis']]

In [685]:
bad_hist = [['no significant medical history'],
 ['no significant medical history'],
 ['no significant medical history'],
 ['dialysis haemo', 'iddm', 'crf', 'heart problem', 'high cholesterol'],
 ['fibromyalgia'],
 ['enlarged liver'],
 ['anxiety', 'autistic', 'adhd', 'depression', 'asthma', 'celiac'],
 ['no significant medical history'],
 ['hypithyroid'],
 ['anxiety'],
 ['no significant medical history'],
 ['hypetension'],
 ['arthritis', 'copd'],
 ['itp', 'wiscott aldridge syndrome', 'eczema'],
 ['suicidal attempts', 'depression'],
 ['high cholesterol', 'hypertension', 'niddm', 'cabg', 'knee surgery years'],
 ['sepsis',
  'urinary tract infection',
  'ca lungs',
  'ca bronchus',
  'hypothyroid',
  'glaucoma cataract',
  'cataract repair',
  'kidney',
  'anemia'],
 ['alcoholism'],
 ['gerd', 'high cholesterol'],
 ['hypertension',
  'tia',
  'osteoporosis',
  'diverticulitis',
  'high cholesterol',
  'anxiety',
  'depression',
  'aortic stenosis']]

In [223]:
#gonna copy the first stage of h diagnoses and overwrite the new words with correct spellings
#using h arbitrarily here...
h_diagnoses_old = h_diagnoses.copy()

In [227]:
#this will go through a list of diagnoses, pull out the first word, see if the first word is in the
#dictionary fo alternate spellings for a word and replace it with the best spelling

for i in range(len(h_diagnoses)):
    word_list = h_diagnoses[i].split()
    
    if word_list[0] in h_spell_dict.keys():
        best_word = get_best_spelling(word_list[0], h_spell_dict[word_list[0]])
        
        if word_list[0] != best_word: 
            word_list[0] = best_word
    
    h_diagnoses[i] = ' '.join(word_list)

   

In [199]:
print(len(h_diagnoses))
print (len(set(h_diagnoses)))

1113
982


In [200]:
#this is the same length as the dict above but the first word has been corrected in many cases
h_diagnoses[:30]

['h amputee',
 'h burn',
 'h foot',
 'h fracture',
 'h infection',
 'h injury',
 'h numbness',
 'h pain',
 'h piloro',
 'h plyori',
 'h plyorie',
 'h pylori',
 'h pyloric',
 'h pylorie',
 'h surgery',
 'h sx',
 'h years',
 'h1n1',
 'ha d',
 'hallucinations',
 'hemangioma',
 'haemhorroids',
 'hemodialysis',
 'hemophilia',
 'hemorrhoids',
 'hemorrhoids problem',
 'hemorrhoids',
 'hair loss',
 'hair procedure',
 'hairline fracture']

In [228]:
#but it hasnt shortened that much
len(Counter(h_diagnoses)), len(Counter(h_diagnoses_old))

(975, 1113)

running the function again to see if it changes the corrections at all...

In [229]:
h_counter2 = (Counter(h_diagnoses))
print (len(h_counter2))
%time h_diagnoses2, h_spell_dict2 = first_letter_counter('h', h_counter2, 5)

975
CPU times: user 1min 17s, sys: 286 ms, total: 1min 17s
Wall time: 1min 19s


In [230]:
#and it doesnt
len(Counter(h_diagnoses2))

975

need to figureout a way to overwrite the original split history, not just the counter...

#### now we are going to see if we can correct whole diagnoses

In [235]:
h_counter2.most_common(50)

[('hemorrhoids', 29),
 ('hypertension', 17),
 ('hypothyroid', 17),
 ('hysterectomy', 13),
 ('hydrocephalus', 9),
 ('hernia', 6),
 ('hypoglycemia', 5),
 ('hemodialysis', 4),
 ('hemophilia', 4),
 ('hepatitis b', 4),
 ('himotos disease', 4),
 ('hydronephrosis', 4),
 ('hyperemesis', 4),
 ('hallucinations', 3),
 ('hemangioma', 3),
 ('himotos', 3),
 ('hypokalemia', 3),
 ('hysterectomy years', 3),
 ('heart attack', 2),
 ('head', 2),
 ('headache', 2),
 ('heart stents', 2),
 ('heart valve', 2),
 ('hemodialysis m', 2),
 ('hemorrhoids sx', 2),
 ('hemorrhoidectomy', 2),
 ('hepatitis', 2),
 ('hepatic b', 2),
 ('hernia repair', 2),
 ('hernia use', 2),
 ('hernia user', 2),
 ('hidradenitis suppurativa', 2),
 ('huntingtons chorea', 2),
 ('hyper', 2),
 ('hyper lipidemia', 2),
 ('hyperacidity', 2),
 ('hysterectomy last', 2),
 ('hysterectomy in', 2),
 ('hysterectomy on', 2),
 ('h amputee', 1),
 ('h burn', 1),
 ('h foot', 1),
 ('h fracture', 1),
 ('h infection', 1),
 ('h injury', 1),
 ('h numbness', 1),
 (

In [237]:
poss_words('h pylori')

['h pylori',
 'h pyloric',
 'h piloro',
 'h pylorie',
 'h plyori',
 'h plyorie',
 'pylori',
 'hpylori',
 'hp pyloric',
 'hel pylori']

In [232]:
poss_words('hysterectomy')

['hystrectomy',
 'hysterrectomy',
 'histerectomy',
 'hyperectomy',
 'hysterectomuy',
 'hystprectomy',
 'hysterectomy',
 'hestorectomy',
 'hyserectomy',
 'hyterectmomy',
 'hyterectomy',
 'hysterctomy']

In [233]:
poss_words('heart attack')

['hert attack',
 'hear attack',
 'heart attack',
 'heart attacked',
 'heartattack',
 'heart attacks',
 'heaart attack']

very promising as it finds lots of similar diagnoses

In [177]:
def whole_diagnosis_counter(diagnosis_counter = diagnoses):
    
    spell_dict = {}
    for diagnosis in list(diagnosis_counter.keys()):
        #print (diagnosis)
        poss_spells = poss_words(diagnosis)
        #print (poss_spells)
        word_count = dict()
        for spell in poss_spells:
            #print (spell)
            word_count[spell] = diagnosis_counter[spell]
        spell_dict[diagnosis] = word_count
        
    return spell_dict

In [178]:
list(h_counter.keys())[:50]

['h amputee',
 'h burn',
 'h foot',
 'h fracture',
 'h infection',
 'h injury',
 'h numbness',
 'h pain',
 'h piloro',
 'h plyori',
 'h plyorie',
 'h pylori',
 'h pyloric',
 'h pylorie',
 'h surgery',
 'h sx',
 'h years',
 'h1n1',
 'ha d',
 'hallucinations',
 'hemangioma',
 'hemorroids',
 'hemodialysis',
 'hemophilia',
 'hemorrhoids',
 'hemorrhoids problem',
 'hair loss',
 'hair procedure',
 'hairline fracture',
 'half of',
 'haliperdol',
 'hallucinations nyd',
 'hallucinations post',
 'hamilton hall',
 'hamstring strain',
 'hamstring tare',
 'hamstring tear',
 'harada disease',
 'hard of',
 'hard on',
 'haringtons',
 'hartmann reversal',
 'hay fever',
 'hb low',
 'hbd',
 'hbh',
 'hbv',
 'hcl',
 'hctz',
 'hd tuethusat']

In [179]:
%time spell_dict = whole_diagnosis_counter(h_counter)

CPU times: user 3min 57s, sys: 873 ms, total: 3min 57s
Wall time: 4min 5s


In [184]:
for k,v in spell_dict.items():
    opt = get_best_spelling(k,v)
    if v != opt:  #only showing the ones where there could be a change in spelling
        print (k,':')
        print (v)
        print (get_best_spelling(k,v))
        print ()

h amputee :
{'h amputee': 1, 'amputee': 0}
h amputee

h burn :
{'h burn': 1, 'burn': 0}
h burn

h foot :
{'h foot': 1, 'foot': 0}
h foot

h fracture :
{'t fracture': 0, 'c fracture': 0, 'h fracture': 1, 'fracture': 0, 'hip fracture': 1}
h fracture

h infection :
{'infection': 0, 'h infection': 1}
h infection

h injury :
{'injury': 0, 'no injury': 0, 'hip injury': 1, 'h injury': 1}
h injury

h numbness :
{'h numbness': 1}
h numbness

h pain :
{'hip pain': 1, 'h pain': 1, 'pain': 0}
h pain

h piloro :
{'h pylori': 1, 'h piloro': 1}
h piloro

h plyori :
{'h pylori': 1, 'h pyloric': 1, 'h pylorie': 1, 'h plyori': 1, 'h plyorie': 1, 'hpylori': 1}
h plyori

h plyorie :
{'h pylori': 1, 'h pyloric': 1, 'h pylorie': 1, 'h plyori': 1, 'h plyorie': 1}
h plyorie

h pylori :
{'h pylori': 1, 'h pyloric': 1, 'h piloro': 1, 'h pylorie': 1, 'h plyori': 1, 'h plyorie': 1, 'pylori': 0, 'hpylori': 1, 'hp pyloric': 1, 'hel pylori': 1}
h pylori

h pyloric :
{'h pylori': 1, 'h pyloric': 1, 'h pylorie': 1, 'h

hernia repair

hernia repir :
{'hernia repairs': 1, 'hernia repar': 1, 'hurnia repair': 0, 'hernia repair': 2, 'hernia repir': 1}
hernia repair

hernia sgx :
{'hernia sx': 1, 'hernia sgx': 1}
hernia sgx

hernia surg :
{'hernia surg': 1}
hernia surg

hernia surgery :
{'hernia surgery': 1}
hernia surgery

hernia surgeryx2 :
{'hernia surgery': 1, 'hernia surgeryx2': 1}
hernia surgeryx2

hernia sx :
{'hernia on': 1, 'hernia sx': 1, 'hernia at': 1, 'hernia in': 1, 'hernias': 0, 'hernia to': 1, 'hernia sgx': 1}
hernia sx

hernia to :
{'hernia on': 1, 'hernia sx': 1, 'hernia at': 1, 'hernia in': 1, 'hernia to': 1}
hernia to

hernia umbilical :
{'hernia umbilical': 1}
hernia umbilical

hernia with :
{'hernia with': 1}
hernia with

herniated belly :
{'herniated belly': 1}
herniated belly

herniated diaphragm :
{'herniated diaphragm': 1}
herniated diaphragm

herniated dics :
{'herinated disc': 1, 'herniated disk': 1, 'herniated dics': 1, 'hernated disc': 1, 'herniates disc': 1, 'herniated discs'

homeopathic cough :
{'homeopathic cough': 1}
homeopathic cough

homeopathic for :
{'homeopathic for': 1}
homeopathic for

homicidal :
{'homicidal': 1}
homicidal

honey combed :
{'honey combed': 1}
honey combed

hyothyroidism :
{'hypothyroidsim': 5, 'hyothyroidism': 1, 'hypothyroidis': 0, 'hypthyroidism': 0, 'hopythyroidism': 0, 'hypothyridism': 0, 'hyptothyroidism': 0, 'hypthyrodism': 0, 'hypothyroidism': 0, 'hypothyroidisn': 0, 'hypothyroidsm': 0, 'hypothroidism': 0, 'hypothyrodism': 1, 'hypothyroidim': 0}
hypothyroidsim

hormonal acne :
{'hormonal acne': 1}
hormonal acne

hormonal imbalance :
{'hormonal imbalance': 1, 'hormone imbalance': 1}
hormonal imbalance

hormonal issues :
{'hormonal issues': 1}
hormonal issues

hormonal replacement :
{'hormonal replacement': 1, 'hormone replacement': 1}
hormonal replacement

hormone deficeincy :
{'hormone deficeincy': 1}
hormone deficeincy

hormone imbalance :
{'hormonal imbalance': 1, 'hormone imbalance': 1}
hormone imbalance

hormone problem

In [185]:
#'let'_clollected_words is made from the keys of a counter 
#so there should not be (and is not) a differnece in these numbers
len(h_collected_words), len(set(h_collected_words))

(939, 939)

In [186]:
h_collected_words[:10]

['h amputee',
 'h burn',
 'h foot',
 'h fracture',
 'h infection',
 'h injury',
 'h numbness',
 'h pain',
 'h piloro',
 'h plyori']

In [187]:
sorted(spell_dict.keys())[:10]

['h amputee',
 'h burn',
 'h foot',
 'h fracture',
 'h infection',
 'h injury',
 'h numbness',
 'h pain',
 'h piloro',
 'h plyori']

In [188]:
#this is my function for overwriting diagnoses with their correct spellings
for i in range(len(h_collected_words)):
    try:
        h_collected_words[i] = get_best_spelling(spell_dict[h_collected_words[i]])
    except:
        continue
    


In [189]:
h_collected_words[:20]

['h amputee',
 'h burn',
 'h foot',
 'h fracture',
 'h infection',
 'h injury',
 'h numbness',
 'h pain',
 'h piloro',
 'h plyori',
 'h plyorie',
 'h pylori',
 'h pyloric',
 'h pylorie',
 'h surgery',
 'h sx',
 'h years',
 'h1n1',
 'ha d',
 'hair loss']

In [190]:
len(set(h_collected_words))

939

### some work that I will not pursue on using only the first word of the diagnosis

In [1343]:
def first_word_counter(counter):
    first_words = []
    for word in counter.keys():
        first_word =  word.split()[0]
        first_words.append(first_word) 

    fw_counter = Counter(first_words)
    
    return fw_counter

In [1344]:
h_fw_counter = first_word_counter(new_h_counter)

In [1345]:
len(h_fw_counter)

313

In [1346]:
h_fw_counter.most_common()

[('heart', 61),
 ('high', 39),
 ('hip', 31),
 ('history', 27),
 ('htn', 20),
 ('hernia', 15),
 ('hysterectomy', 13),
 ('hearing', 11),
 ('hypertension', 10),
 ('hand', 9),
 ('hep', 9),
 ('hepatic', 9),
 ('heavy', 8),
 ('hemiplegia', 7),
 ('hiv', 7),
 ('head', 6),
 ('hemorrhoids', 6),
 ('herniated', 6),
 ('home', 6),
 ('headache', 5),
 ('heel', 5),
 ('hypertrophic', 5),
 ('healthy', 4),
 ('hemangioma', 4),
 ('hereditary', 4),
 ('herpes', 4),
 ('hypo', 4),
 ('holter', 4),
 ('hormonal', 4),
 ('hormone', 4),
 ('hyperactive', 4),
 ('hyperplasia', 4),
 ('h', 3),
 ('hear', 3),
 ('hallucinations', 3),
 ('hemodialysis', 3),
 ('hematuria', 3),
 ('hepatitis', 3),
 ('hgb', 3),
 ('hodgkins', 3),
 ('hospitalized', 3),
 ('htnn', 3),
 ('huntingtons', 3),
 ('hydrocele', 3),
 ('hydrocephalus', 3),
 ('hyperemesis', 3),
 ('hypoglycemia', 3),
 ('h-pylori', 2),
 ('hamstring', 2),
 ('he', 2),
 ('healed', 2),
 ('hematoma', 2),
 ('hemicolectomy', 2),
 ('hemophilia', 2),
 ('hemorrhagic', 2),
 ('her', 2),
 ('her

now we are going to collapse the diagnoses to only care about the first word...

In [132]:
h_first_words = []
for word in h_counter.keys():
    first_word =  word.split()[0]
    h_first_words.append(first_word) 
        
h_fw_counter = Counter(h_first_words)

In [133]:
#much shorter but a lot of information is lost
len(h_fw_counter)

316

In [134]:
h_fw_counter.most_common()[100:120]

[('hemorroids', 1),
 ('hairline', 1),
 ('half', 1),
 ('haliperdol', 1),
 ('hamilton', 1),
 ('harada', 1),
 ('haringtons', 1),
 ('hartmann', 1),
 ('hay', 1),
 ('hb', 1),
 ('hbd', 1),
 ('hbh', 1),
 ('hbv', 1),
 ('hcl', 1),
 ('hctz', 1),
 ('hd', 1),
 ('hdhd', 1),
 ('headinjury', 1),
 ('heaing', 1),
 ('heal', 1)]

In [135]:
sorted(h_counter.most_common()[100:120])

[('hctz', 1),
 ('hd tuethusat', 1),
 ('hdhd', 1),
 ('he a', 1),
 ('he does', 1),
 ('head ache', 1),
 ('head concusion', 1),
 ('head fracture', 1),
 ('head injjury', 1),
 ('head injuries', 1),
 ('head injury', 1),
 ('head neck', 1),
 ('head pain', 1),
 ('head surgery', 1),
 ('head sx', 1),
 ('head trauma', 1),
 ('head tumor', 1),
 ('head tumour', 1),
 ('headache concussion', 1),
 ('heart faillure', 1)]

In [346]:
#what happens if you make all the items with the same first word are considered the same?
first_words = []
for word in c_counter.keys():
    first_word =  word.split()[0]
    first_words.append(first_word) 

In [347]:
first_words[20:30]

['c-palate',
 'c-palate',
 'c-palate',
 'c-pap',
 'c-pap',
 'c-sction',
 'c-sec',
 'c-sect',
 'c-sect',
 'c-sect']

In [348]:
#this is promising.  decreases the number of diagnoses in half
fw_counter = Counter(first_words); len(fw_counter)

643

### some regex experimentation

In [238]:
hp = poss_words('h pylori')

In [239]:
hp

['h pylori',
 'h pyloric',
 'h piloro',
 'h pylorie',
 'h plyori',
 'h plyorie',
 'pylori',
 'hpylori',
 'hp pyloric',
 'hel pylori']

In [242]:
for h in hp:
    print (re.match('\b[a-z]\s[a-z]*', hp))

TypeError: expected string or bytes-like object

In [247]:
s = re.sub(r"([a-z])\s([a-z])", r"\1-\2", hp[0]) 

In [248]:
s

'h-pylori'

In [251]:
s = re.sub(r"([a-z])\s([a-z])", r"\1-\2", 'a fib'); s

'a-fib'

In [265]:
s = re.sub(r"([a-z])\s([a-z])", r"\1-\2", 'h pylori'); s

'h-pylori'

In [275]:
s = re.sub(r"(\b[a-z]\b)\s([a-z])", r"\1-\2", 'h pr'); s

'h-pr'